In [1]:
import numpy as np
import os
import pandas as pd
import PyPDF2
import re
import pprint as pp

In [2]:
path = r"C:\Users\Chris\OneDrive\Documents\Projects\Bank_Statement_Scrape\data\PNC"
regex = r"^\d{2}/\d{2} \d*,?\d*\.\d{2} .+"

In [3]:
files = os.listdir(path)
matches = []

for file in files:
    year = file.split("\\")[-1].split("-")[0]
    with open(path + "\\" + file, "rb") as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page in reader.pages:
            text = text + page.extract_text(0)
            text = text + '\n'

    found = re.findall(regex, text, flags=re.MULTILINE)
    for x in found:
        matches.append('/'.join([year, x]))

In [4]:
clean_data = []

for line in matches:
    date, amount, *recip = line.split()
    date = date
    amount = amount.replace(',', '')
    if amount.startswith('.'):
        amount = '0' + amount
    recip = ' '.join(recip)
    clean_data.append([date] + ['Discover'] + [amount] + [recip])

In [5]:
data_types = {
    'Account': object,
    'Amount': float,
    'Recipient': object,
    'Category': object,
    'SubCategory': object,
    'Project': object,
    'Note': object
}

In [6]:
df = pd.DataFrame(clean_data, columns=['Date', 'Account', 'Amount', 'Recipient'])
df['Category'] = np.NaN
df['SubCategory'] = np.NaN
df['Project'] = np.NaN
df['Note'] = np.NaN

df = df.astype(data_types)
df.set_index('Date', inplace=True)
df.to_excel('pnc_transactions.xlsx')